# 🚀 PQC Certificate Chain Launcher with Real Dilithium Support
This notebook attempts to use real **Dilithium** keys via `pyoqs`, falling back to EC keys if PQC support isn't available.

> Ensure you're using Google Colab with internet access and sufficient RAM.

In [ ]:
# ✅ Step 1: Install Dependencies (Cryptography + PQC Tools)
!pip install cryptography
!pip install pyoqs || echo 'pyoqs unavailable, falling back to EC keys.'

In [ ]:
# 🧠 Step 2: Import Utility Functions
import sys
sys.path.append('./src')
from cert_utils import create_placeholder_certificate, build_certificate_chain
try:
    from pyoqs import Signature
    OQS_AVAILABLE = True
except ImportError:
    print("pyoqs not available. Falling back to EC keys.")
    OQS_AVAILABLE = False

In [ ]:
# 🔐 Step 3: Generate Keypairs (Dilithium or EC fallback)
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import ec

def generate_keypair():
    if OQS_AVAILABLE:
        sig = Signature("Dilithium3")
        public_key = sig.generate_keypair()
        return sig, public_key
    else:
        key = ec.generate_private_key(ec.SECP384R1(), default_backend())
        return key, key.public_key()

# Use real or placeholder keys
root_key, root_pub = generate_keypair()
inter_key, inter_pub = generate_keypair()
user_key, user_pub = generate_keypair()

In [ ]:
# 🔏 Step 4: Create Placeholder Certificates (Using keys above)
root_cert = create_placeholder_certificate("Root CA", "Root CA", root_key)
inter_cert = create_placeholder_certificate("Intermediate CA", "Root CA", inter_key)
user_cert = create_placeholder_certificate("leaf.com", "Intermediate CA", user_key)

In [ ]:
# 💾 Step 5: Save PEM Files
with open("root.pem", "wb") as f:
    f.write(root_cert.public_bytes(serialization.Encoding.PEM))
with open("intermediate.pem", "wb") as f:
    f.write(inter_cert.public_bytes(serialization.Encoding.PEM))
with open("user.pem", "wb") as f:
    f.write(user_cert.public_bytes(serialization.Encoding.PEM))

In [ ]:
# 🔗 Step 6: Load and Print Certificate Chain
chain = build_certificate_chain("user.pem", ["intermediate.pem"], "root.pem")
for idx, cert in enumerate(chain):
    print(f"\nCertificate {idx+1}:")
    print(cert.subject)